In [1]:
pip install requests beautifulsoup4


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
from bs4 import BeautifulSoup
import time
import sqlite3

In [4]:
def setup_database(db_name="github_google_repos.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

In [6]:
def setup_database(db_name="github_google_repos.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    # テーブルの作成
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS repositories (
            id INTEGER PRIMARY KEY,
            repo_name TEXT NOT NULL UNIQUE,
            language TEXT,
            stars INTEGER
        )
    """)
    conn.commit()
    return conn

In [9]:
def scrape_and_save(conn):
    url = "https://github.com/orgs/google/repositories"
    print(f"スクレイピング: {url}")

In [15]:
def save_data_to_db(conn, data_list):
    """

    :param conn: データベース接続オブジェクト
    :param data_list: (repo_name, language, stars) のタプルのリスト
    """
    cursor = conn.cursor()

    sql_insert = """
        INSERT OR IGNORE INTO repositories (repo_name, language, stars)
        VALUES (?, ?, ?)
    """
    
    # リスト内のデータ一つ一つに対してSQLを実行
    for repo_name, language, stars in data_list:
        try:
            cursor.execute(sql_insert, (repo_name, language, stars))
            
        except Exception as e:
            print(f"データの挿入に失敗しました ({repo_name}): {e}")

    conn.commit()
    print(f" {len(data_list)}件のデータをデータベースに保存しました。")

In [ ]:
import sqlite3

def display_data(conn):
    """
    
    :param conn: データベース接続オブジェクト
    """
    print("\n--- データベースの結果（SELECT文） ---")
    
    cursor = conn.cursor()
    
    cursor.execute("""
        SELECT 
            repo_name, 
            language, 
            stars 
        FROM 
            repositories 
        ORDER BY 
            stars DESC
    """)
    
    # すべての結果を取得
    results = cursor.fetchall()
    
    if not results:
        print("⚠️ データがありません。スクレイピングが成功しているか確認してください。")
        return
    
    print("-" * 65)
    print(f"{'リポジトリ名':<40} {'主要言語':<15} {'スター数':>10}")
    print("-" * 65)
    
    # 結果の表示
    for name, lang, stars in results:
        print(f"{name:<40} {lang:<15} {stars:>10,}")
    
    print("-" * 65)
    